## Паттерн Наблюдатель

Паттерн позволяет создать механизм, позволяющий другим объектам подписываться на определенные события и реагировать на них

В данном примере есть класс EspManager c методами прикрепления / открепления нового подписчика и методом уведомления подписчиков о событии

Есть класс Esp, описывающий работы насоса с методом, позволяющим рассчитать подачу насоса

И есть несколько классов подписчиков - абстрактный работник (Worker), а также Operator и Master. Отдельно взятым операторам и мастерам интересны определенные события, на которые они подписываются и у них есть метод react, описывающий их реакцию на эти события

In [1]:
import pandas as pd
from abc import ABC, abstractmethod

In [3]:
class EspManager:
    
    def __init__(self, event_types=None):
        
        if event_types is None:
            # Поддерживаемые типы событий по умолчанию
            event_types = ['Насос или датчик сгорел', 'Срыв подачи']
            
        # Подписчики на события, происходящие с ЭЦН
        self._subscribers = {event: [] for event in event_types}
        
    def subscribe(self, subscriber, event_type):
        """
        Прикрепление нового подписчика на событие
        """
        if event_type in self._subscribers.keys():
            self._subscribers[event_type].append(subscriber)
    
    def unsubscribe(self, subscriber, event_type):
        """
        Открепление существующего подписчика на событие
        """
        if event_type in self._subscribers.keys():
            self._subscribers[event_type].remove(subscriber)
    
    def notify(self, event_type):
        """
        Уведомление подписчиков подписанных на событие
        """
        if event_type in self._subscribers.keys():
            for subscriber in self._subscribers[event_type]:
                subscriber.react(event_type)

In [4]:
class Esp:
    
    def __init__(self, q_max=100):
        self.manager = EspManager()
        self.q_max = q_max
    
    @property
    def subscribe(self):
        return self.manager.subscribe
    
    @property
    def unsubscribe(self):
        return self.manager.unsubscribe
    
    def calc_rate(self, p):
        """
        Определяем сколько жидкости течет через насос
        """
        if p == 0:
            self.manager.notify('Насос или датчик сгорел')
            return
        else:
            q = 2 * p
        
        if q > self.q_max:
            self.manager.notify('Срыв подачи')
        return

In [5]:
class Worker(ABC):
    
    @abstractmethod
    def react(self):
        pass

In [6]:
class Operator(Worker):
    
    def __init__(self, name):
        self.name = name
        
    def react(self, event_type):
        
        print(f'{event_type}. Оператор {self.name} едет на скважину')
        
        if event_type.lower() == 'насос или датчик сгорел':
            self._get_pump()

    def _get_pump(self,):
        print(f'Оператор {self.name} достает насос из скважины')

In [7]:
class Master(Worker):
    def __init__(self, name, operators):
        self.name = name
        self.operators = operators 
    
    def react(self, event_type):
        print(f'{event_type}. Мастер {self.name} зол.')
        
        for operator in self.operators:
            operator.react(event_type)

In [8]:
operator1 = Operator('Коля')
operator2 = Operator('Саня')
boss = Master('Вася', operators=[operator2])

esp = Esp(q_max=50)

In [9]:
esp.subscribe(operator1, 'Срыв подачи')
esp.subscribe(operator2, 'Срыв подачи')
esp.subscribe(boss, 'Насос или датчик сгорел')

In [10]:
esp.calc_rate(50)

Срыв подачи. Оператор Коля едет на скважину
Срыв подачи. Оператор Саня едет на скважину


In [11]:
esp.calc_rate(0)

Насос или датчик сгорел. Мастер Вася зол.
Насос или датчик сгорел. Оператор Саня едет на скважину
Оператор Саня достает насос из скважины


In [12]:
esp.manager.unsubscribe(boss, 'Насос или датчик сгорел')

In [13]:
esp.calc_rate(0)

In [14]:
esp.calc_rate(50)

Срыв подачи. Оператор Коля едет на скважину
Срыв подачи. Оператор Саня едет на скважину
